前処理

In [ ]:
import pandas as pd
from datetime import datetime, time

columns = ["着順", "枠番", "馬番", "馬名", "性齢", "斤量", "騎手", "タイム", "着差", "単勝", "人気", "馬体重", "調教師"]
year_list = ["2023", "2022", "2021", "2020", "2019", "2018", "2017", "2016", "2015", "2014"]

merged_df = pd.DataFrame(columns=columns)

for year in year_list:
    print(year)
    file_path = "../data/result/" + year + "_data.csv"
    df = pd.read_csv(file_path, names=columns)
    merged_df = pd.concat([merged_df, df])

#データの整形
merged_df[['性別', '年齢']] = merged_df['性齢'].str.extract(r'([牡牝])(\d+)')
merged_df["性別"].replace(["牡", "牝"], [0, 1], inplace=True)
merged_df[['出走前体重', '出走後変化']] = merged_df['馬体重'].str.extract(r'(\d+)\(([-+]?\d+)\)')
values_to_remove = ['中', '取', '除', '4(降)', '5(降)']
merged_df = merged_df[~merged_df['着順'].isin(values_to_remove)]
merged_df.drop(columns=['馬番', '性齢', '馬体重', '着差', '調教師'], inplace=True)
#欠損値埋め
merged_df['タイム'].fillna(-1, inplace=True)
merged_df['性別'].fillna(-1, inplace=True)
merged_df['年齢'].fillna(-1, inplace=True)
merged_df['出走前体重'].fillna(480, inplace=True)
merged_df['出走後変化'].fillna(0, inplace=True)
#型変換
merged_df['年齢'] = merged_df['年齢'].astype(int)
merged_df['出走前体重'] = merged_df['出走前体重'].astype(int)
merged_df['出走後変化'] = merged_df['出走後変化'].astype(int)
merged_df['着順'] = merged_df['着順'].astype(int)
merged_df['枠番'] = merged_df['枠番'].astype(int)
merged_df['単勝'] = merged_df['単勝'].astype(float)
merged_df['タイム'] = merged_df['タイム'].apply(lambda x: datetime.strptime(x, '%M:%S.%f').time() if isinstance(x, str) else x)

CSV出力

In [ ]:
merged_df.to_csv('data.csv', encoding='utf-8', index=False)